In [4]:
from spotify_wrapper import SpotifyWrapper
from database_wrapper import SpotifyCharts

# make it so that when this cell is run, update the libraries and modifications
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
%reload_ext autoreload

In [26]:
# Load the YAML secrets
import yaml
with open("secrets.yaml", 'r') as stream:
    try:
        secrets = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

# Create the SpotifyWrapper object
sw = SpotifyWrapper(secrets['CLIENT_ID'], secrets['CLIENT_SECRET'], secrets["APP_NAME"], secrets["REDIRECT_URI"], secrets["SCOPE"])


In [7]:
data_path = "../data/charts.csv"

# Create the SpotifyCharts object
sc = SpotifyCharts(data_path)

In [5]:
sc

SpotifyCharts(path=../data/charts.csv, columns=Index(['title', 'rank', 'date', 'artist', 'url', 'region', 'chart', 'trend',
       'streams'],
      dtype='object'), shape=(26173514, 9))

# Utils functions, to be moved later on

In [27]:
def get_region_audio_features(sw : SpotifyWrapper, sc: SpotifyCharts, region_names, date_range = None):
    """
    Get a audio features of a data limited with a region
    """
    # Get the data
    region_data = sc.get_by_region(region_names)

    # if date range is not None, filter the data
    if date_range is not None:
        region_data = region_data[region_data["date"].between(date_range[0], date_range[1])]

    # Get the audio features
    track_ids = region_data["url"].apply(lambda x: x.split("/")[-1])
    audio_features = sw.get_audio_features(track_ids)
    return audio_features

# get_region_audio_features(sw, sc, ["Turkey"], date_range = ("2020-01-01", "2020-04-10"))

def get_data_by_region_date(sw : SpotifyWrapper, sc : SpotifyCharts, region_names, date_range = None):
    """
    Get a audio features of a data limited with a region
    """
    # Get the data
    region_data = sc.get_by_region(region_names)

    # if date range is not None, filter the data
    if date_range is not None:
        region_data = region_data[region_data["date"].between(date_range[0], date_range[1])]

    return region_data

_ds = get_data_by_region_date(sw, sc, ["Germany"], date_range = ("2020-01-01", "2020-01-07"))
_ds[_ds["chart"] == 'viral50']

,title,rank,date,artist,url,region,chart,trend,streams
18737588,How Dare You,1,2020-01-01,Derex,https://open.spotify.com/track/6FngZCqvzNE4lAi...,Germany,viral50,SAME_POSITION,NaN
18737589,GOODMORNINGTOKYO!,2,2020-01-01,TOKYO’S REVENGE,https://open.spotify.com/track/0Q2n5yzl2XRqYbV...,Germany,viral50,SAME_POSITION,NaN
18737590,Blinding Lights,3,2020-01-01,The Weeknd,https://open.spotify.com/track/0sf12qNH5qcw8qp...,Germany,viral50,SAME_POSITION,NaN
18737591,Falling,4,2020-01-01,Trevor Daniel,https://open.spotify.com/track/4TnjEaWOeW0eKTK...,Germany,viral50,SAME_POSITION,NaN
18737592,Roses - Imanbek Remix,5,2020-01-01,SAINt JHN,https://open.spotify.com/track/24Yi9hE78yPEbZ4...,Germany,viral50,SAME_POSITION,NaN
...,...,...,...,...,...,...,...,...,...
18907093,Bad Guy (Remix),46,2020-01-07,Dani L. Mebius,https://open.spotify.com/track/6uGiIWcBzSYqO5Z...,Germany,viral50,MOVE_DOWN,NaN
18907094,Close Your Eyes,47,2020-01-07,"Felix Jaehn, VIZE, Miss Li",https://open.spotify.com/track/7N06jo3EnV792VF...,Germany,viral50,MOVE_UP,NaN
18907095,Reason to Get Lost,48,2020-01-07,ArtLec,https://open.spotify.com/track/5X8vNnFg4XKL6Wk...,Germany,viral50,NEW_ENTRY,NaN
18907096,Eternal Youth,49,2020-01-07,RŮDE,https://open.spotify.com/track/0ytvsZOerGzUWfH...,Germany,viral50,NEW_ENTRY,NaN


In [28]:
import pandas as pd
import tqdm
import time
import requests
from spotipy import SpotifyException
# Get the unique title, artist pairs
ds = sc.df
sp = sw.sp

# Get the unique URLs
unique_urls = ds["url"].unique()

# Split '\' and get the last element
unique_track_ids = [url.split("/")[-1] for url in unique_urls]

def get_audio_features(track_ids):
    try:
        return sp.audio_features(track_ids)
    except  SpotifyException as e:
        # Wait for 2 seconds before retrying
        # print("Rate limit reached, waiting for a minute ")
        # time.sleep(60)
        print(e)
        return get_audio_features(track_ids)

if len(unique_track_ids) > 100:
    splitted = [unique_track_ids[i:i + 100] for i in range(0, len(unique_track_ids), 100)]

    audio_features = []
    # approximately 180 requests per minute can be made so we can wait 0.33 seconds between each request
    for i in tqdm.tqdm(range(len(splitted))):
        if i < 2000:
            continue
        time.sleep(0.33)
        # Use the get_audio_features function which handles 429 errors
        audio_features_batch = get_audio_features(splitted[i])
        audio_features.extend(audio_features_batch)

# Create a dataframe for audio features and URL
audio_features_df = pd.DataFrame(audio_features)
audio_features_df["url"] = unique_urls

# # Save data 
# audio_features_df.to_csv("../data/audio_features_ct.csv", index=False)


100%|██████████| 2178/2178 [01:42<00:00, 21.20it/s] 


AttributeError: 'NoneType' object has no attribute 'keys'

In [31]:
# remove none value
audio_features = [x for x in audio_features if x is not None]

In [16]:
# Create a dataframe for audio features and URL
audio_features_df = pd.DataFrame(audio_features)
audio_features_df["url"] = unique_track_ids[:audio_features_df.shape[0]]

# Save data 
audio_features_df.to_csv("../data/audio_features_2.csv", index=False)

AttributeError: 'NoneType' object has no attribute 'keys'

In [20]:
# Drop the nones in audio_features list
audio_features = [x for x in audio_features if x is not None]

In [32]:
audio_features[0]

{'danceability': 0.609,
 'energy': 0.512,
 'key': 11,
 'loudness': -12.081,
 'mode': 0,
 'speechiness': 0.0288,
 'acousticness': 0.39,
 'instrumentalness': 0.0042,
 'liveness': 0.0655,
 'valence': 0.0778,
 'tempo': 118.008,
 'type': 'audio_features',
 'id': '6FIvxqDq1Sc39uwqELQTpr',
 'uri': 'spotify:track:6FIvxqDq1Sc39uwqELQTpr',
 'track_href': 'https://api.spotify.com/v1/tracks/6FIvxqDq1Sc39uwqELQTpr',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6FIvxqDq1Sc39uwqELQTpr',
 'duration_ms': 223739,
 'time_signature': 4}

In [33]:
audio_features_df = pd.DataFrame(audio_features)

# get track_id from track_href .split("/")[-1]
audio_features_df["track_id"] = audio_features_df["track_href"].apply(lambda x: x.split("/")[-1])


In [35]:
audio_features_df["track_id"].nunique()

17702

In [36]:
audio_features_df.to_csv("../data/audio_features_2.csv", index=False)

In [37]:
df1 = pd.read_csv("../data/audio_features.csv")
df2 = pd.read_csv("../data/audio_features_2.csv")

# concatanate the rows
df = pd.concat([df1, df2])

# eliminate duplicates with track_id
df = df.drop_duplicates(subset=['track_id'])

df.to_csv("../data/audio_features_processed.csv", index=False)

In [44]:
audio_features_df


,0
0,None
1,None
2,None
3,None
4,None
5,None
6,None
7,None
8,None
9,None


In [45]:
# from unique track ids, find which track id is not in df["track_id"]

# Get the unique URLs
unique_urls = ds["url"].unique()

# Split '\' and get the last element
unique_track_ids = [url.split("/")[-1] for url in unique_urls]

# get the track ids that are not in df["track_id"]
track_ids_not_in_df = list(set(unique_track_ids) - set(df["track_id"].unique()))

# get the urls that are not in df["url"]
print(track_ids_not_in_df)

# explicitly add them to df by using spotify api to get all the audio features

audio_features = get_audio_features(track_ids_not_in_df)

# Create a dataframe for audio features and URL
audio_features_df = pd.DataFrame(audio_features)

# get track_id from track_href .split("/")[-1]
audio_features_df["track_id"] = audio_features_df["track_href"].apply(lambda x: x.split("/")[-1])

# concatanate the rows
df = pd.concat([df, audio_features_df])

df.to_csv("../data/audio_features_processed_last.csv", index=False)


['6DCg2lVxo5goz5aC2w82D9', '7sFnb9vK5q0wlfAQRUZITs', '2DEYFawpGha5Zn54Fx6dX5', '4IjwENtDPy4KqpnU9iBBca', '2EqVEnDcVA0qlk52agyGw6', '50Ar6Q1bnWjKGaDHGQTOsk', '0vcQZpthiccMvMlYv2YPzd', '37E2Ld7fZIl9NjVlvlO9OU', '5VKJr18j5O5Bl4KeYrBH0K', '1baW36rUsrJ4H5qj9AYrDa', '3aiLkdJbT9V51m2GAqfn4r', '7eJchyg35TI5g1bVUwGH1A', '3uMIfxV8q5tPTWzJh8s6Mk']


KeyError: 'track_href'

In [47]:
sp.track(track_id="6DCg2lVxo5goz5aC2w82D9")

# get the audio features of the track
sp.audio_features(tracks=["6DCg2lVxo5goz5aC2w82D9"])


[None]